In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale


import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("D:/dataset/cleaned_improved_cicids2017.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler

tmp = df.drop(labels = 'Label',axis=1)
labels = df['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(tmp)

X_scaled_df = pd.DataFrame(X_scaled, columns=tmp.columns)  # 스케일된 데이터를 DataFrame으로 변환
df_merged = pd.concat([X_scaled_df, labels.reset_index(drop=True)], axis=1)  # Index 정렬 후 병합

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_merged, test_size=0.1, random_state=42)
X_train.shape, X_test.shape

In [ ]:
X_train_normal = X_train[X_train['Label'] == 0]
X_train_normal.shape

X_test_normal = X_test[X_test['Label'] == 0]
X_test_normal.shape

In [ ]:
X_train_not_normal = X_train[X_train['Label'] !=0]

X_test = pd.concat([X_test, X_train_not_normal])

In [ ]:
y_test = X_test['Label']
X_test = X_test.drop(labels='Label',axis=1)
y_train_normal = X_train_normal['Label']
X_train_normal = X_train_normal.drop(labels='Label', axis=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector, Conv1D, Conv1DTranspose
from keras.models import Model
from keras import regularizers
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.callbacks import EarlyStopping

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model, Input
import numpy as np
from tensorflow.keras import backend as K

latent_dim = 10
inter_dim = 20

# Sampling function for reparameterization trick
def sampling(args):
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    return z_mean + z_log_sigma * epsilon

def vae_loss(x, x_decoded_mean, z_mean, z_log_sigma):
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean), axis=1)
    kl_loss = -0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    kl_loss_weighted = kl_loss * 0.001  # KL 다이버전스 손실에 가중치 부여
    total_loss = K.mean(reconstruction_loss + kl_loss_weighted)
    return total_loss

def cvae(X, labels):
    features = X.shape[1]  # 2D 데이터이므로 timesteps는 필요 없고, features만 사용
    input_x = Input(shape=(features,), name='InputFeatures')  # 2D 입력 (샘플, 특징)

    input_label = Input(shape=(1,), name='InputLabel')  # Assumes binary or multi-class label

    # Embed the label into the same shape as the input
    embedded_label = layers.Embedding(input_dim=2, output_dim=features)(input_label)  # Adjust output_dim to match features
    embedded_label = layers.Flatten()(embedded_label)

    # Concatenate input_x and embedded_label
    concatenated_input = layers.Concatenate()([input_x, embedded_label])

    # Reshape for Conv1D layer (to 3D)
    reshaped_input = layers.Reshape((features + embedded_label.shape[-1], 1))(concatenated_input)

    # Encoder with CNN layers
    h = layers.Conv1D(filters=64, kernel_size=3, activation="relu", padding='same')(reshaped_input)
    h = layers.Conv1D(filters=32, kernel_size=3, activation="relu", padding='same')(h)
    h = layers.Conv1D(filters=16, kernel_size=3, activation="relu", padding='same')(h)

    # Flatten for dense layers (MLP)
    h = layers.Flatten()(h)
    h = layers.Dense(inter_dim, activation='relu')(h)
    h = layers.Dense(inter_dim, activation='relu')(h)

    # z layer
    z_mean = layers.Dense(latent_dim)(h)
    z_log_sigma = layers.Dense(latent_dim)(h)
    z = layers.Lambda(sampling)([z_mean, z_log_sigma])

    # Decoder
    # Add the label to the latent space z
    z_with_label = layers.Concatenate()([z, embedded_label])

    # Expand for Conv1DTranspose layer
    decoder1 = layers.Dense((features + embedded_label.shape[-1]) * inter_dim)(z_with_label)
    decoder1 = layers.Reshape((features + embedded_label.shape[-1], inter_dim))(decoder1)

    # Decoder with CNN layers
    decoder1 = layers.Conv1DTranspose(filters=16, kernel_size=3, activation="relu", padding='same')(decoder1)
    decoder1 = layers.Conv1DTranspose(filters=32, kernel_size=3, activation="relu", padding='same')(decoder1)
    decoder1 = layers.Conv1DTranspose(filters=64, kernel_size=3, activation="relu", padding='same')(decoder1)

    # Flatten and final dense layer to reconstruct original features
    decoder1 = layers.Flatten()(decoder1)
    decoder1 = layers.Dense(features)(decoder1)
    
    model = Model([input_x, input_label], decoder1)
    model.add_loss(vae_loss(input_x, decoder1, z_mean, z_log_sigma))
    
    return model

# Create CVAE model
model = cvae(X_train_normal, y_train_normal)
model.summary()

from tensorflow.keras.callbacks import EarlyStopping
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])
# Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile and train the model with early stopping
history = model.fit([X_train_normal, y_train_normal], X_train_normal,
                    shuffle=True,
                    epochs=50, 
                    validation_split=0.1,  
                    batch_size=32,
                    callbacks=[early_stopping]).history

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model, Input
import numpy as np
from tensorflow.keras import backend as K

latent_dim = 10  # Latent 공간 차원
inter_dim = 20   # 중간 층 차원

# Sampling function for reparameterization trick
def sampling(args):
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    return z_mean + z_log_sigma * epsilon

# VAE 손실 함수 정의
def vae_loss(x, x_decoded_mean, z_mean, z_log_sigma):
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean), axis=1)  # MSE 기반 복원 손실
    kl_loss = -0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)  # KL 다이버전스
    kl_loss_weighted = kl_loss * 0.0001  # KL 손실 가중치 적용
    total_loss = K.mean(reconstruction_loss + kl_loss_weighted)
    return total_loss

# VAE 모델 생성
def vae(X_train):
    features = X_train.shape[1]  # 입력 차원
    input_x = Input(shape=(features,), name='InputFeatures')  # 2D 입력 (샘플, 특징)

    # Encoder
    h = layers.Dense(128, activation='relu')(input_x)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(inter_dim, activation='relu')(h)

    # Latent space
    z_mean = layers.Dense(latent_dim)(h)
    z_log_sigma = layers.Dense(latent_dim)(h)
    z = layers.Lambda(sampling)([z_mean, z_log_sigma])

    # Decoder
    decoder = layers.Dense(inter_dim, activation='relu')(z)
    decoder = layers.Dense(64, activation='relu')(decoder)
    decoder = layers.Dense(128, activation='relu')(decoder)
    decoder = layers.Dense(features, activation='linear')(decoder)  # 원래 feature 차원으로 복원

    model = Model(input_x, decoder)
    model.add_loss(vae_loss(input_x, decoder, z_mean, z_log_sigma))  # VAE Loss 추가

    return model

# VAE 모델 생성
model = vae(X_train_normal)
model.summary()

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))

# EarlyStopping 콜백
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train_normal, X_train_normal,
                    shuffle=True,
                    epochs=100,
                    validation_split=0.1,
                    batch_size=32,
                    callbacks=[early_stopping]).history

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# VAE 및 CVAE에서 추출한 latent space 벡터 (예제 데이터)
z_vae = np.random.randn(5000, 10)  # (5000개 샘플, 10차원 잠재 공간)
z_cvae = np.random.randn(5000, 10) + 2  # CVAE는 특정 방향으로 분포가 이동할 가능성이 있음

# 샘플링 (속도 개선)
num_samples = 1000
indices = np.random.choice(len(z_vae), num_samples, replace=False)
z_vae_sampled = z_vae[indices]
z_cvae_sampled = z_cvae[indices]

# PCA 변환
pca_vae = PCA(n_components=2).fit_transform(z_vae_sampled)
pca_cvae = PCA(n_components=2).fit_transform(z_cvae_sampled)

# t-SNE 변환
tsne_vae = TSNE(n_components=2, perplexity=20, n_iter=500, random_state=42).fit_transform(z_vae_sampled)
tsne_cvae = TSNE(n_components=2, perplexity=20, n_iter=500, random_state=42).fit_transform(z_cvae_sampled)

# UMAP 변환
umap_vae = umap.UMAP(n_components=2, random_state=42).fit_transform(z_vae_sampled)
umap_cvae = umap.UMAP(n_components=2, random_state=42).fit_transform(z_cvae_sampled)

# 시각화
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

axs[0].scatter(pca_vae[:, 0], pca_vae[:, 1], alpha=0.5, label="VAE")
axs[0].scatter(pca_cvae[:, 0], pca_cvae[:, 1], alpha=0.5, label="CVAE")
axs[0].set_title("PCA")

axs[1].scatter(tsne_vae[:, 0], tsne_vae[:, 1], alpha=0.5, label="VAE")
axs[1].scatter(tsne_cvae[:, 0], tsne_cvae[:, 1], alpha=0.5, label="CVAE")
axs[1].set_title("t-SNE")

axs[2].scatter(umap_vae[:, 0], umap_vae[:, 1], alpha=0.5, label="VAE")
axs[2].scatter(umap_cvae[:, 0], umap_cvae[:, 1], alpha=0.5, label="CVAE")
axs[2].set_title("UMAP")

for ax in axs:
    ax.legend()

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 예제 데이터: VAE 및 CVAE의 정상 데이터 및 이상 데이터 Reconstruction Error
recon_error_vae_normal = np.random.normal(loc=0.02, scale=0.005, size=500)
recon_error_vae_anomaly = np.random.normal(loc=0.06, scale=0.01, size=100)

recon_error_cvae_normal = np.random.normal(loc=0.01, scale=0.004, size=500)
recon_error_cvae_anomaly = np.random.normal(loc=0.08, scale=0.012, size=100)

# Box Plot으로 비교
plt.figure(figsize=(10, 6))
plt.boxplot([
    recon_error_vae_normal, recon_error_vae_anomaly,
    recon_error_cvae_normal, recon_error_cvae_anomaly
], labels=["VAE Normal", "VAE Anomaly", "CVAE Normal", "CVAE Anomaly"])

plt.title("Reconstruction Error Comparison (VAE vs CVAE)")
plt.ylabel("Reconstruction Error")
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

# K-Means 클러스터링
kmeans_vae = KMeans(n_clusters=3, random_state=42).fit(z_vae_sampled)
kmeans_cvae = KMeans(n_clusters=3, random_state=42).fit(z_cvae_sampled)

# DBSCAN 클러스터링
dbscan_vae = DBSCAN(eps=0.5, min_samples=5).fit(z_vae_sampled)
dbscan_cvae = DBSCAN(eps=0.5, min_samples=5).fit(z_cvae_sampled)

# 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(z_vae_sampled[:, 0], z_vae_sampled[:, 1], c=kmeans_vae.labels_, cmap="viridis", alpha=0.5)
plt.title("K-Means Clustering on VAE Latent Space")

plt.subplot(1, 2, 2)
plt.scatter(z_cvae_sampled[:, 0], z_cvae_sampled[:, 1], c=kmeans_cvae.labels_, cmap="viridis", alpha=0.5)
plt.title("K-Means Clustering on CVAE Latent Space")
plt.show()